In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
from matbench.bench import MatbenchBenchmark
from matbench.constants import CLF_KEY
from gptchem.gpt_regressor import GPTRegressor
from gptchem.tuner import Tuner

from loguru import logger
import matplotlib.pyplot as plt
logger.enable("gptchem")

In [3]:
import decorator, time

def retry(howmany, *exception_types, **kwargs):
    timeout = kwargs.get('timeout', 0.0) # seconds
    @decorator.decorator
    def tryIt(func, *fargs, **fkwargs):
        for _ in range(howmany):
            try: return func(*fargs, **fkwargs)
            except exception_types or Exception as e:
                print(e)
                if timeout is not None: time.sleep(timeout)
    return tryIt

In [4]:
mb = MatbenchBenchmark(
    autoload=True,
    subset=[
      #  "matbench_expt_gap",
        "matbench_steels",
    ],
)

2023-01-29 16:50:44 INFO     Initialized benchmark 'matbench_v0.1' with 1 tasks: 
['matbench_steels']


In [5]:
@retry(3, timeout=5)
def train_test_fold(task, fold):
    regressor = GPTRegressor(task.metadata['target'], Tuner(n_epochs=8, learning_rate_multiplier=0.02, wandb_sync=False))
    train_inputs, train_outputs = task.get_train_and_val_data(fold)

    # train and validate your model
    regressor.fit(train_inputs, train_outputs.values)

    # Get testing data
    test_inputs = task.get_test_data(fold, include_target=False)

    # Predict on the testing data
    # Your output should be a pandas series, numpy array, or python iterable
    # where the array elements are floats or bools
    predictions = regressor.predict(test_inputs)

    # Record your data!
    task.record(fold, predictions)
    return predictions

In [6]:
from collections import defaultdict

In [7]:
predictions = defaultdict(list)

for task in mb.tasks:
    task.load()
    
    for fold_ind, fold in enumerate(task.folds):
        if task.is_recorded[fold_ind]:
           print(f"Skipping fold {fold_ind} of {task.dataset_name}")
           continue
        pred = train_test_fold(task, fold)
        predictions[task.dataset_name].append(pred)
        train_inputs, train_outputs = task.get_train_and_val_data(fold)

    #print(f"{task.dataset_name}: MAE  {task.scores['mae']['mean']}")

2023-01-29 16:50:45 INFO     Dataset matbench_steels already loaded; not reloading dataset.


Upload progress: 100%|██████████| 78.5k/78.5k [00:00<00:00, 106Mit/s]


Uploaded file from /Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230129_165045/train.jsonl: file-ZnEkED360c14K5E1vcqZNoKj


2023-01-29 16:50:47.194 | DEBUG    | gptchem.tuner:tune:188 - Requested fine tuning. {
  "created_at": 1675007447,
  "events": [
    {
      "created_at": 1675007447,
      "level": "info",
      "message": "Created fine-tune: ft-rMYckfLuWYHcbvr7qYbGJXH0",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": 0.02,
    "n_epochs": 8,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-rMYckfLuWYHcbvr7qYbGJXH0",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-TFRJXw3PPQocOWbu71eI2t9U",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 78515,
      "created_at": 1675007446,
      "filename": "/Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230129_165045/train.jsonl",
      "id": "file-ZnEkED360c14K5E1vcqZNoKj",
      "object": "file",
      "purpose": "fine-tune",
      "status": "uploaded",
      "s

2023-01-29 17:00:55 INFO     Recorded fold matbench_steels-0 successfully.


Upload progress: 100%|██████████| 78.5k/78.5k [00:00<00:00, 64.0Mit/s]


Uploaded file from /Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230129_170055/train.jsonl: file-g6PY76lglFQqfaJDdwGJbyTt


2023-01-29 17:00:59.176 | DEBUG    | gptchem.tuner:tune:188 - Requested fine tuning. {
  "created_at": 1675008059,
  "events": [
    {
      "created_at": 1675008059,
      "level": "info",
      "message": "Created fine-tune: ft-pf6czb1EYkgUkafrFya8Ep0X",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": 0.02,
    "n_epochs": 8,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-pf6czb1EYkgUkafrFya8Ep0X",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-TFRJXw3PPQocOWbu71eI2t9U",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 78453,
      "created_at": 1675008057,
      "filename": "/Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230129_170055/train.jsonl",
      "id": "file-g6PY76lglFQqfaJDdwGJbyTt",
      "object": "file",
      "purpose": "fine-tune",
      "status": "uploaded",
      "s

In [ ]:
mb.to_file("gpt_bench.json.gz")